# Transfer Learning for Mining Feature Requests and Bug Reports from Tweets and App Store Reviews

Demo notebook of the paper: **Transfer Learning for Mining Feature Requests and Bug Reports from Tweets and App Store Reviews** presented at the conference  *AIRE 2021*.

## Usage

* Uncomment the models that  you want to test in the variable *models_to_test* in the section: *User Configuration Variables*

* Add the sentences that you want to test into the variables: *sentences_to_predict_tweet_english, sentences_to_predict_app_review_english and sentences_to_predict_tweet_italian* in the section: *User Configuration Variables*

* Run all the cells in the notebook

## User Configuration Variables

In [9]:
#Uncomment the models that you want to test
models_to_test = {
                  'bert-base-uncased-app_review-inq-en':('bert-base-uncased','10SOt1n96X00PHHhD025mJ6OYzZpeInCA','app_review','en','inq'),
                  'bert-base-uncased-app_review-irr-en':('bert-base-uncased','10j9dPT_SNDVaQhSwnDFpz0NKb9kKgNPR','app_review','en','irr'),
                  'bert-base-uncased-app_review-pbr-en':('bert-base-uncased','11A9aYJzaiN1j42RM8G3e0RxhebDj0yTZ','app_review','en','pbr'),

                  'bert-base-uncased-tweet-inq-en': ('bert-base-uncased','11RMszEsz9iG6oJJfVi6_o57BGb3sxaby','tweet','en','inq'),
                  'bert-base-uncased-tweet-irr-en': ('bert-base-uncased','11eHMbr32JHJe7Xaea3TEw-FcK6MNC_lG','tweet','en','irr'),
                  'bert-base-uncased-tweet-pbr-en': ('bert-base-uncased','11zEI0vFc228EkZZEuEd0QTwpjFPXr6T6','tweet','en','pbr'),

                  'bert-base-italian-cased-tweet-inq-it': ('dbmdz/bert-base-italian-cased','12b4XeVch6ltWutQW-nBy0eMKhb7bpQdq','tweet','it','inq'),
                  'bert-base-italian-cased-tweet-irr-it': ('dbmdz/bert-base-italian-cased','131y7r0VcACPRmD3G0EDjUuyAglqSINUt','tweet','it','irr'),
                  'bert-base-italian-cased-tweet-pbr-it': ('dbmdz/bert-base-italian-cased','12U0uA6geUt9wMgQsWg5M0_K9Zp2jRJ2C','tweet','it','pbr'),
                  }

In [16]:
#Add the sentences that you want to predict
sentences_to_predict_tweet_english = [
                                      "@Company how do I change my config?",
                                      "@Company My Wifi is not working, even though I already restarted it three times!",
                                      "@Company Very good customer service"
                                      ]

sentences_to_predict_app_review_english = [
                                           "How Can I logout from the app?",
                                           "The donwload button in the menu is not working",
                                           "Great app. 5 stars with no doubt"
                                          ]

sentences_to_predict_tweet_italian = [
                                      '@Company come posso controllare il mio saldo nella pagina web?',
                                      '@Company Il mio wifi è lentissimo oggi',
                                      "@Company l'impiegato che mi ha aiutato ieri ha fatto un ottimo lavoro. Vi ringrazio molto!"
                                      ]

## Prepare Environment

### Donwload Models and Install Dependencies

In [3]:
#Donwload models
models_PATH = '/tmp/models/'
!mkdir $models_PATH

for model_name, model_info in models_to_test.items():
  checkpoint_id = model_info[1]
  path_to_donwload = models_PATH + model_name + '.ckpt'
  !gdown --id $checkpoint_id -O $path_to_donwload

#Install dependencies
!pip install transformers==4.6.1
!pip install pytorch-lightning==1.3.5

Downloading...
From: https://drive.google.com/uc?id=10SOt1n96X00PHHhD025mJ6OYzZpeInCA
To: /tmp/models/bert-base-uncased-app_review-inq-en.ckpt
1.31GB [00:08, 160MB/s]
Downloading...
From: https://drive.google.com/uc?id=10j9dPT_SNDVaQhSwnDFpz0NKb9kKgNPR
To: /tmp/models/bert-base-uncased-app_review-irr-en.ckpt
1.31GB [00:07, 164MB/s]
Downloading...
From: https://drive.google.com/uc?id=11A9aYJzaiN1j42RM8G3e0RxhebDj0yTZ
To: /tmp/models/bert-base-uncased-app_review-pbr-en.ckpt
1.31GB [00:10, 126MB/s] 
Downloading...
From: https://drive.google.com/uc?id=11RMszEsz9iG6oJJfVi6_o57BGb3sxaby
To: /tmp/models/bert-base-uncased-tweet-inq-en.ckpt
1.31GB [00:09, 132MB/s]
Downloading...
From: https://drive.google.com/uc?id=11eHMbr32JHJe7Xaea3TEw-FcK6MNC_lG
To: /tmp/models/bert-base-uncased-tweet-irr-en.ckpt
1.31GB [00:09, 135MB/s]
Downloading...
From: https://drive.google.com/uc?id=11zEI0vFc228EkZZEuEd0QTwpjFPXr6T6
To: /tmp/models/bert-base-uncased-tweet-pbr-en.ckpt
1.31GB [00:10, 129MB/s] 
Downloading

### Helper Classes

In [4]:
#Helper classes
import pytorch_lightning as pl
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn as nn
import numpy as np
import abc
 
from re import T
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_fscore_support
import pandas as pd
 
class CustomModel(pl.LightningModule):
 
    def __init__(self, hyperparams, training_dataset, validation_dataset, labels, model_to_use, tokenizer, training_sampler):
        super().__init__()
 
        self.hyperparams = hyperparams
        self.training_dataset = training_dataset
        self.validation_dataset = validation_dataset
        self.labels = labels
        self.tokenizer = tokenizer
        self.define_model(model_to_use, tokenizer)
        self.training_sampler = training_sampler
 
        self.best_f1 = 0
        self.epoch_best_f1 = 0
 
    @abc.abstractmethod
    def define_model(self, model_to_use, tokenizer):
        pass
 
    @abc.abstractmethod
    def forward(self, ids, mask, token_type_ids):
        pass
 
    def loss_fn(self, outputs, targets):
      return torch.nn.CrossEntropyLoss()(outputs, targets)
 
    def general_step(self, batch, batch_idx, mode):
      ids = batch['ids']
      mask = batch['mask']
      token_type_ids = batch['token_type_ids']
      targets = batch['targets']
 
      outputs = self.forward(ids, mask, token_type_ids)
 
      return {'outputs': outputs, 'targets': targets}
 
    #******Training******
    #This method runs on each GPU
    def training_step(self, batch, batch_idx):
      return self.general_step(batch, batch_idx, "train")
    
    #This method aggregates the results of training_step in the different GPUs
    def training_step_end(self, aggregated_outputs):
      loss = self.loss_fn(aggregated_outputs["outputs"], aggregated_outputs["targets"])
      self.log('training_loss',loss)
      return {'loss':loss}
    
    #This method runs at the end of each epoch
    def training_epoch_end(self, results_of_each_batch):
      pass
 
    #******Validation******
    #This method runs in each GPU
    def validation_step(self, batch, batch_idx):
      return self.general_step(batch, batch_idx, "val")
 
    #This method aggregates the results of validation_step in the different GPUs
    def validation_step_end (self, aggregated_outputs):
      outputs = torch.tensor(aggregated_outputs['outputs'].cpu().detach().numpy().tolist())
      prediction_values, prediction_indexes = torch.max(outputs, dim=1)
      targets = aggregated_outputs['targets'].cpu().detach().numpy()
 
      return {'predictions': prediction_indexes, 'targets': targets}
 
    #This method runs at the end of each epoch
    def validation_epoch_end(self, results_of_each_batch):
      targets = np.empty([0])
      predictions = np.empty([0])
      
      for result in results_of_each_batch:
        targets = np.concatenate((targets,result['targets']))
        predictions = np.concatenate((predictions,result['predictions']))
      
      accuracy = accuracy_score(targets, predictions)
      precision, recall, f1, _ = precision_recall_fscore_support(targets, 
                                                                 predictions, 
                                                                 average="binary", 
                                                                 pos_label=1,
                                                                 zero_division=0)
      auc_score = 0
      try:
        auc_score = roc_auc_score(targets, predictions)
      except:
        pass
 
      results = { 'accuracy':accuracy,
                  'precision':precision,
                  'recall':recall,
                  'f1':f1,
                  'auc':auc_score
                }
 
      self.log('f1', f1)
 
      if f1 > self.best_f1:
        self.best_f1 = f1
        self.epoch_best_f1 = self.current_epoch
 
      print(f'epoch {self.current_epoch}: {results}')
 
    def configure_optimizers(self):
      return torch.optim.Adam(params = self.parameters(), lr=self.hyperparams["learning_rate"])
 
    #******Dataloaders******
    def train_dataloader(self):
      if self.training_sampler is None:
        shuffle = self.hyperparams["shuffle"]
      else:
        shuffle = False
      
      return DataLoader(self.training_dataset, 
                        batch_size=self.hyperparams["train_batch_size"], 
                        shuffle=shuffle, 
                        num_workers=2, 
                        sampler=self.training_sampler )
 
    def val_dataloader(self):
      return DataLoader(self.validation_dataset, 
                        batch_size=self.hyperparams["validation_batch_size"], 
                        shuffle= False, 
                        num_workers=2)
      
from transformers import BertTokenizer, BertModel, BertConfig
import transformers
import torch
 
class BERTCustomModel(CustomModel):
 
    def define_model(self, model_to_use, tokenizer):
      self.l1 = transformers.BertModel.from_pretrained(model_to_use)
      self.l1.resize_token_embeddings(len(tokenizer)) 
 
      self.l2 = torch.nn.Dropout(0.3)
      self.l3 = torch.nn.Linear(768, len(self.labels))
 
    def forward(self, ids, mask, token_type_ids):
 
      transformer_output = self.l1(input_ids = ids, token_type_ids = token_type_ids, attention_mask= mask)
      hidden_state = transformer_output.last_hidden_state
      
      output_1 = hidden_state[:, 0] #CLS token
      output_2 = self.l2(output_1)
      output = self.l3(output_2)
      
      return output

def tokenize_sentence(sentence, tokenizer):

  max_len = 200

  inputs = tokenizer.encode_plus(
      sentence,
      None,
      add_special_tokens=True,
      max_length=max_len,
      padding='max_length',
      return_token_type_ids=True,
      truncation = True
  )
  ids = inputs['input_ids']
  mask = inputs['attention_mask']
  token_type_ids = inputs["token_type_ids"]

  return {
            'ids': torch.tensor(ids, dtype=torch.long).unsqueeze(0),
            'mask': torch.tensor(mask, dtype=torch.long).unsqueeze(0),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long).unsqueeze(0),
        }
  
def process_sentences(sentences, langage, source):

  for sentence in sentences:

    print(f'Sentence to predict: {sentence}')

    for model_name, model_info in models.items():
      model = model_info[0]
      model_source = model_info[1] 
      model_language = model_info[2]
      tokenizer = model_info[3]
      category = model_info[4]
      
      if model_language == langage and model_source == source:
        prediction = predict_sentence(sentence, model, tokenizer)
        print(f'{category}: ->: {prediction.item()}')
    print('')

def predict_sentence(sentence, model, tokenizer):
  
  batch = tokenize_sentence(sentence, tokenizer)

  ids = batch['ids']
  mask = batch['mask']
  token_type_ids = batch['token_type_ids']

  output = model.forward(ids, mask, token_type_ids)

  _, prediction = torch.max(output, dim=1)

  return prediction

### Load Models From the Donwloaded Checkpoints

In [5]:
models = {}

for model_name, model_info in models_to_test.items():

  bert_model_to_use = model_info[0]
  checkpoint_name = model_name + '.ckpt'
  checkpoint_path = models_PATH + checkpoint_name

  source = model_info[2]
  language = model_info[3]
  category = model_info[4]

  tokenizer = BertTokenizer.from_pretrained(bert_model_to_use)
  tokenizer.add_tokens("@mention",special_tokens=False)

  #Load models
  model = BERTCustomModel.load_from_checkpoint(hyperparams = None,
                                               training_dataset = None, 
                                               validation_dataset = None, 
                                               labels = [0,1], 
                                               model_to_use = bert_model_to_use,
                                               tokenizer=tokenizer,
                                               training_sampler=None,
                                               checkpoint_path = checkpoint_path)
  
  models[model_name] = (model,source,language,tokenizer,category)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.

Some weights of the model checkpoint at dbmdz/bert-base-italian-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at dbmdz/bert-base-italian-cased were not used when initializing BertModel: ['cls.seq_relationship.weight'

## Predict Sentences

In [17]:
#Predict sentences
print('**************************')
print('Tweet English')
print('**************************')
process_sentences(sentences_to_predict_tweet_english,'en','tweet')
print('**************************')
print('App Review English')
print('**************************')
process_sentences(sentences_to_predict_app_review_english,'en','app_review')
print('**************************')
print('Tweet Italian')
print('**************************')
process_sentences(sentences_to_predict_tweet_italian,'it','tweet')

**************************
Tweet English
**************************
Sentence to predict: @Company how do I change my config?
inq: ->: 1
irr: ->: 0
pbr: ->: 0

Sentence to predict: @Company My Wifi is not working, even though I already restarted it three times!
inq: ->: 0
irr: ->: 0
pbr: ->: 1

Sentence to predict: @Company Very good customer service
inq: ->: 0
irr: ->: 1
pbr: ->: 0

**************************
App Review English
**************************
Sentence to predict: How Can I logout from the app?
inq: ->: 1
irr: ->: 1
pbr: ->: 0

Sentence to predict: The donwload button in the menu is not working
inq: ->: 0
irr: ->: 0
pbr: ->: 1

Sentence to predict: Great app. 5 stars with no doubt
inq: ->: 0
irr: ->: 1
pbr: ->: 0

**************************
Tweet Italian
**************************
Sentence to predict: @Company come posso controllare il mio saldo nella pagina web?
inq: ->: 1
irr: ->: 0
pbr: ->: 0

Sentence to predict: @Company Il mio wifi è lentissimo oggi
inq: ->: 0
irr: ->: